<a href="https://colab.research.google.com/github/henriquevedoveli/TCC/blob/main/notebooks/tccHyperTuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Otimização de Hiper-Parâmetros das Redes Neurais**

---

## **Classificação de Pneumonia em Radiografia do Tórax Utilizando Redes Neurais Convolucionais**

---

#### Autor: Henrique Hundsdorfer Vedoveli

#### Orientador: Prof. Dr. Anuar José Mincache

#### Coorientador: Prof. Dr. Breno Ferraz de Oliveira

---


Neste notebook foi utilizado o algoritimo _HyperBand_ para a otimização do número de neurônios e a taxa de aprendizado do otimizador _Adam_.

Os valores obtidos foram:

|              | Número de Neurônios | Taxa de Aprendizado |
|:------------:|:-------------------:|:-------------------:|
|    _LeNet-5_   |         430         |        1e-04        |
|    _AlexNet_   |         840         |        1e-05        |
|    _VGG-16_    |         840         |        1e-05        |
| _Inception-V3_ |          -          |        1e-05        |
|   _ResNet-50_  |          -          |                     |
|    _VGG-Inc_  |         2680        |        1e-05        |

---



---
# Bibliotecas Necessárias
---

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!pip install -q -U keras-tuner

################################################################################
 
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import model_selection, metrics
 
import tensorflow as tf
from tensorflow import keras
from keras.layers import Conv2D, Convolution2D, MaxPooling2D, AveragePooling2D, 
from keras.layers import concatenate, Flatten, Dense,  Dropout, Input
from keras.models import Sequential, Model, load_model
from tensorflow.keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.preprocessing import image  
from keras.preprocessing.image import ImageDataGenerator

import keras_tuner as kt
 
########################################################################################
import warnings
warnings.filterwarnings("ignore")

pd.set_option('display.float_format', lambda x: '%.3f' % x)

np.random.seed(7)
 
import os
 
%matplotlib inline

---
## Local onde as imagens estão armazenadas
---

In [ ]:
TRAIN = '/content/drive/MyDrive/TCC/data/chest_xray/train'
TEST = '/content/drive/MyDrive/TCC/data/chest_xray/test'
VAL = '/content/drive/MyDrive/TCC/data/chest_xray/val'

MODELS = '/content/drive/MyDrive/TCC/MODELS'

---

# Carregando as imagens e aplicando o _data augmentation_

----

## Parâmetros utilizados para o _data augmentation_

|                        Distorção                       |  Parâmetro |
|:------------------------------------------------------:|:----------:|
|           Alteração na Escala _(Zoom Range)_           | 0.9 a 1,25 |
|           Faixa de Rotação _(Rotation Range)_          |     10°    |
| Faixa de Deslocamento Horizontal _(Width Shift Range)_ |     0.1    |
|  Faixa de Deslocamento Vertical _(Height Shift Range)_ |     0.1    |
|           Modo de Preenchimento _(Fill Mode)_          |  Constante |
|                          Cval                          |      0     |

In [ ]:
trainData = ImageDataGenerator(zoom_range = [0.9, 1.25], 
                                        rotation_range = 10,
                                        vertical_flip = True,
                                        # horizontal_flip = True, 
                                        width_shift_range = 0.1, 
                                        height_shift_range = 0.1,
                                        brightness_range = [0.5, 1],
                                        shear_range = .7,
                                        fill_mode='constant', 
                                        cval=0)

trainData = trainData.flow_from_directory(directory=TRAIN,
                                       batch_size = 32,
                                       target_size=(224,224),
                                       color_mode = 'grayscale')

valData = ImageDataGenerator()
valData = valData.flow_from_directory(
        VAL,
        target_size = (224,224),
        batch_size = 32,
        color_mode = 'grayscale'
        )

Found 5082 images belonging to 3 classes.
Found 150 images belonging to 3 classes.


---
# Otimização _LeNet-5_
---
## Construindo e compilando o modelo 

In [ ]:
def modelBuilderLeNet(hp):
    hpUnits = hp.Int('units', min_value=10, max_value=1000, step=10)

    hpLearningRate = hp.Choice('learning_rate', values=[1e-1, 1e-2, 1e-3, 1e-4, 1e-5])

    modelLeNet = Sequential()

    modelLeNet.add(Conv2D(filters=6, kernel_size = (5,5), activation='relu', input_shape=(32, 32, 1)))
    modelLeNet.add(AveragePooling2D(2,2))

    modelLeNet.add(Conv2D(filters=16, kernel_size = (5,5), activation='relu'))
    modelLeNet.add(AveragePooling2D(2,2))

    modelLeNet.add(Conv2D(filters=120, kernel_size = (5,5), activation='relu'))

    modelLeNet.add(Flatten())

    modelLeNet.add(Dense(units=hpUnits, activation='tanh'))
    modelLeNet.add(Dense(units=3, activation='softmax'))


    modelLeNet.compile(loss='categorical_crossentropy', 
                     optimizer=keras.optimizers.Adam(learning_rate=hpLearningRate),
                     metrics = ['accuracy'])

    return modelLeNet

## Iniciando o algoritmo de otimização e iniciando a pesquisa ao melhores hiper-parâmetros

In [ ]:
tunerLeNet = kt.Hyperband(modelBuilderLeNet,
                     objective='val_accuracy',
                     max_epochs=60,
                     factor=5,
                     directory='/content/drive/MyDrive/TCC',
                     project_name='TCC_Hypertuning_LeNet')

In [ ]:
tunerLeNet.search(trainData, validation_data = valData, epochs=10)

best_hps=tunerLeNet.get_best_hyperparameters(num_trials=1)[0]

---
# Otimização _AlexNet_ 
---
## Construindo e compilando o modelo 

In [ ]:
def modelBuilderAlexNet(hp):
    hpUnits = hp.Int('units', min_value=100, max_value=5000, step=10)

    hpLearningRate = hp.Choice('learning_rate', values=[1e-1, 1e-2, 1e-3, 1e-4, 1e-5])

    modelAlex = Sequential()

    modelAlex.add(Conv2D(filters=96, kernel_size = (11,11), activation='relu', input_shape=(224, 224, 1)))
    modelAlex.add(MaxPooling2D(3,3))

    modelAlex.add(Conv2D(filters=256, kernel_size = (5,5), activation='relu'))
    modelAlex.add(MaxPooling2D(3,3))

    modelAlex.add(Conv2D(filters=384, kernel_size = (3,3), activation='relu'))
    modelAlex.add(Conv2D(filters=384, kernel_size = (3,3), activation='relu'))
    modelAlex.add(Conv2D(filters=256, kernel_size = (3,3), activation='relu'))
    modelAlex.add(MaxPooling2D(3,3))

    modelAlex.add(Flatten())

    modelAlex.add(Dense(units=hpUnits, activation='relu'))
    modelAlex.add(Dense(units=hpUnits, activation='relu'))
    modelAlex.add(Dense(units=3, activation='softmax'))


    modelAlex.compile(loss='categorical_crossentropy', 
                     optimizer=keras.optimizers.Adam(learning_rate=hpLearningRate),
                     metrics = ['accuracy'])

    return modelAlex

## Iniciando o algoritmo de otimização e iniciando a pesquisa ao melhores hiper-parâmetros

In [ ]:
tunerAlex = kt.Hyperband(modelBuilderAlexNet,
                     objective='val_accuracy',
                     max_epochs=60,
                     factor=5,
                     directory='/content/drive/MyDrive/TCC',
                     project_name='TCC_Hypertuning_AlexNet')

In [ ]:
tunerAlex.search(trainData, validation_data = valData, epochs=10)

best_hps=tunerAlex.get_best_hyperparameters(num_trials=1)[0]

---
# Otimização _VGG-16_ 
---
## Construindo e compilando o modelo 

In [ ]:
def modelBuilderVGG(hp):
    hpUnits = hp.Int('units', min_value=100, max_value=5000, step=10)

    hpLearningRate = hp.Choice('learning_rate', values=[1e-1, 1e-2, 1e-3, 1e-4, 1e-5])


    modelVGG = tf.keras.applications.VGG16(input_shape=(224, 224, 1),include_top=False, weights=None)

    modelVGG = tf.keras.Sequential([
            modelVGG,
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(hpUnits, activation=tf.nn.relu),
            tf.keras.layers.Dropout(0.2),
            tf.keras.layers.Dense(hpUnits, activation=tf.nn.relu),
            tf.keras.layers.Dropout(0.2),
            tf.keras.layers.Dense(3,activation=tf.nn.softmax)
            ])
    
    modelVGG.compile(loss='categorical_crossentropy', 
                     optimizer=keras.optimizers.Adam(learning_rate=hpLearningRate),
                     metrics = ['accuracy'])

    return modelVGG

## Iniciando o algoritmo de otimização e iniciando a pesquisa ao melhores hiper-parâmetros

In [ ]:
tunerVGG = kt.Hyperband(modelBuilderVGG,
                     objective='val_accuracy',
                     max_epochs=60,
                     factor=5,
                     directory='/content/drive/MyDrive/TCC',
                     project_name='TCC_Hypertuning_VGG')

In [ ]:
stopEarly = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=10)

tunerVGG.search(trainData, validation_data = valData, epochs=10, callbacks=[stopEarly])

best_hps=tunerVGG.get_best_hyperparameters(num_trials=1)[0]

---
# Otimização _Inception-V3_ 
---
## Construindo e compilando o modelo 

## Iniciando o algoritmo de otimização e iniciando a pesquisa ao melhores hiper-parâmetros

---
# Otimização _ResNet-50_ 
---
## Construindo e compilando o modelo 

## Iniciando o algoritmo de otimização e iniciando a pesquisa ao melhores hiper-parâmetros

---
# Otimização _VGG-Inc_
---
## Definição do módulo _inception_

Para a definição do módulo _inception_ é necessário passar como parâmetro o _output_ da camada anterior da rede neural, e o número de filtros de dimensionalidade de cada camada. A função retorna como saída os tensores resultantes de cada convolução concatenado. 

In [ ]:
def inceptionModule(x,
                    filters1x1, 
                    filters3x3r,
                    filters3x3,
                    filters5x5r, 
                    filters5x5,
                    filtersPoll,
                    name=None):
    
    '''
    params: x (tensor) ->  output da camada anterior da cnn

            filters1x1 (int) -> numeros de filtros de dimensionalidade para a camada 
            de convolucao com tamanho de kernel (1x1)

            filters3x3r (int) -> numeros de filtros de dimensionalidade para a camada 
            de convolucao com tamanho de kernel (1x1) que antecede a camada de 
            convolucao de tamanho (3x3)

            filters3x3 (int) -> numeros de filtros de dimensionalidade para a camada 
            de convolucao com tamanho de kernel (3x3)

            filters5x5r (int) -> numeros de filtros de dimensionalidade para a camada 
            de convolucao com tamanho de kernel (1x1) que antecede a camada de 
            convolucao de tamanho (5x5)

            filters5x5 (int) -> numeros de filtros de dimensionalidade para a camada 
            de convolucao com tamanho de kernel (5x5)

            filtersPoll (int) -> numeros de filtros de dimensionalidade para a camada 
            de convolucao que vem apos a camada de pooling com tamanho (1x1)

            name (str) --opicional-- -> nome do modulo 
    '''
    
    conv1x1 = Convolution2D(filters1x1, (1,1),padding='same', activation='relu')(x)
    conv3x3 = Convolution2D(filters3x3r, (1, 1),padding='same', activation='relu')(x)
    conv3x3 = Convolution2D(filters3x3, (3, 3),padding='same', activation='relu')(conv3x3)

    conv5x5 = Convolution2D(filters5x5r, (1, 1),padding='same', activation='relu')(x)
    conv5x5 = Convolution2D(filters5x5, (5, 5),padding='same', activation='relu')(conv5x5)

    poolProj = MaxPooling2D((3, 3), strides=(1, 1), padding='same')(x)
    poolProj = Convolution2D(filtersPoll, (1, 1),padding='same', activation='relu')(poolProj)

    output = concatenate([conv1x1, conv3x3, conv5x5, poolProj], axis=3,name=name)
    
    return output

## Construindo e compilando o modelo 



In [ ]:
def modelBuilderDInc(hp):
    hpUnits = hp.Int('units', min_value=100, max_value=5000, step=10)

    hpLearningRate = hp.Choice('learning_rate', values=[1e-1, 1e-2, 1e-3, 1e-4, 1e-5])

    input_layer = Input(shape=(224, 224, 1))
    x = Convolution2D(64, (3,3), strides=(1, 1), activation='relu')(input_layer)
    x = Convolution2D(64, (3,3), strides=(1, 1), activation='relu')(x)
    x = MaxPooling2D((3, 3), strides=(2, 2))(x)

    #2nd group
    x = Convolution2D(128, (3,3), strides=(1, 1), activation='relu')(x)
    x = Convolution2D(128, (3,3), strides=(1, 1), activation='relu')(x)
    x = MaxPooling2D((3, 3), strides=(2, 2))(x)

    #3rd group
    x = Convolution2D(256, (3,3), strides=(1, 1), activation='relu')(x)
    x = Convolution2D(256, (3,3), strides=(1, 1), activation='relu')(x)
    x = MaxPooling2D((3, 3), strides=(2, 2))(x)

    #4rd group
    x = Convolution2D(512, (3,3), strides=(1, 1), activation='relu')(x)
    x = Convolution2D(512, (3,3), strides=(1, 1), activation='relu')(x)
    x = MaxPooling2D((3, 3), strides=(2, 2))(x)

    x = Convolution2D(512, (3,3), strides=(1, 1), activation='relu')(x)
    x = Convolution2D(512, (3,3), strides=(1, 1), activation='relu')(x)
    x = MaxPooling2D((3, 3), strides=(2, 2))(x)

    x = inceptionModule(x,
                        filters1x1=64,
                        filters3x3r=96,
                        filters3x3=128,
                        filters5x5r=16,
                        filters5x5=32,
                        filtersPoll=32,
                        name='inception1')

    x = inceptionModule(x,
                        filters1x1=128,
                        filters3x3r=128,
                        filters3x3=192,
                        filters5x5r=32,
                        filters5x5=96,
                        filtersPoll=64,
                        name='inception2')
    
    x = inceptionModule(x,
                        filters1x1=192,
                        filters3x3r=95,
                        filters3x3=208,
                        filters5x5r=16,
                        filters5x5=48,
                        filtersPoll=64,
                        name='inception3')

    x = Flatten()(x)
    x = Dropout(0.2)(x)
    x = Dense(hpUnits, activation = 'relu')(x)
    x = Dropout(0.2)(x)
    x = Dense(hpUnits, activation = 'relu')(x)
    x = Dense(3, activation = 'softmax', name='output')(x)

    modelDInc = Model(input_layer, x, name='D-Incep')
    modelDInc.compile(loss='categorical_crossentropy', 
                     optimizer=keras.optimizers.Adam(learning_rate=hpLearningRate),
                     metrics = ['accuracy'])

    return modelDInc

## Iniciando o algoritmo de otimização e iniciando a pesquisa ao melhores hiper-parâmetros

In [ ]:
tunerDINC = kt.Hyperband(modelBuilderDInc,
                     objective='val_accuracy',
                     max_epochs=60,
                     factor=5,
                     directory='/content/drive/MyDrive/TCC',
                     project_name='TCC_Hypertuning_DInc')

INFO:tensorflow:Reloading Oracle from existing project /content/drive/MyDrive/TCC/TCC_Hypertuning_DInc/oracle.json
INFO:tensorflow:Reloading Tuner from /content/drive/MyDrive/TCC/TCC_Hypertuning_DInc/tuner0.json


In [ ]:
stopEarly = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=10)

tunerDINC.search(trainData, validation_data = valData, epochs=10)

best_hps=tunerDINC.get_best_hyperparameters(num_trials=1)[0]